In [895]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup, CData
import requests

from time import sleep
import pandas as pd 
import numpy as np

In [762]:
driver = webdriver.Chrome("/Applications/chromedriver")

In [1478]:
driver.get("https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/accueil.aspx")
page = driver.find_element_by_class_name('divWCAD304_FamilyList')


parent_urls = [elem.get_attribute("href") for elem in page.find_elements_by_tag_name('a')]
parent_urls = parent_urls[2:10]
urls = []
for parent in parent_urls:
    sleep(1)
    driver.get(parent)
    sleep(1)
    try:
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "divWCAD304_FamilyList")))
        page = driver.find_element_by_class_name('divWCAD304_FamilyList')
        a = [elem.get_attribute("href") for elem in page.find_elements_by_tag_name('a')]
        for url in a:
            urls.append(url)
    except:
        next

        
urls
    


['https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284325-Boucherie.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284326-Volailles-et-Gibiers.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284327-Poissonnerie.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-285187-Traiteur-de-la-mer.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284351-Legumes.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284350-Jus-de-fruits-et-legumes-frais.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284349-Fruits.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284358-Pains-et-Patisseries.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284361-Pains-de-mie-et-Pains-grilles.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284362-Patisse

In [1479]:
for url in urls:
    print(url)

https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284325-Boucherie.aspx
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284326-Volailles-et-Gibiers.aspx
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284327-Poissonnerie.aspx
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-285187-Traiteur-de-la-mer.aspx
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284351-Legumes.aspx
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284350-Jus-de-fruits-et-legumes-frais.aspx
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284349-Fruits.aspx
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284358-Pains-et-Patisseries.aspx
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284361-Pains-de-mie-et-Pains-grilles.aspx
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284362-Patisseries-moelleuses.aspx
https://fd4-cours

In [1492]:
product_pages = []

for url in urls:
    driver.get(url)

    shelters = driver.find_element_by_class_name('divWCRS315_FiltresNavigation').find_element_by_css_selector('ul')
    shelter_main = driver.find_element_by_class_name('ulWCAD307_FilAriane').find_elements_by_css_selector('li')[-1].text

    links = [elem.get_attribute("href") for elem in shelters.find_elements_by_tag_name('a')]


    for link in links:
        sleep(1)
        if links.index(link) == 0:
            next
        else:      
            driver.get(link)

            actions = ActionChains(driver)
            actions.move_to_element(driver.find_element_by_id("ulListeProduits").find_elements_by_css_selector('li')[-1]).perform()

            shelter_child = driver.find_element_by_class_name('ulWCAD307_FilAriane').find_elements_by_css_selector('li')[-1].text

            sleep(1)
            request_text = requests.get(link)
            soup = BeautifulSoup(request_text.text,"html.parser" ,from_encoding="utf-8")


            brands = []
            try:
                wait = WebDriverWait(driver, 10)
                wait.until(EC.visibility_of_element_located((By.XPATH, "//dt[contains(text(),'Marques')]")))
                if driver.find_element_by_id('divBlocsFiltres').find_element_by_xpath("//dt[contains(text(),'Marques')]"):
                    for filter in driver.find_elements_by_class_name('dlCRS315_Filtre'):
                        if "Marques" in filter.find_element_by_css_selector("dt").text:
                            for label in filter.find_elements_by_css_selector('label'):
                                brands.append(label.text)
                product_pages.append({"url":link, 
                         "shelter_parent": shelter_main,
                         "shelter_child": shelter_child,
                         "brands":brands,
                         "soup": soup, 
                         "text": driver.find_element_by_id("ulListeProduits").text
                        })
            except:
                product_pages.append({"url":link, 
                     "shelter_parent": shelter_main,
                     "shelter_child": shelter_child,
                     "brands":brands,
                     "soup": soup,
                     "text": driver.find_element_by_id("ulListeProduits").text
                    })


In [1501]:
data = pd.DataFrame(product_pages)
data

,brands,shelter_child,shelter_parent,soup,text,url
0,"[Marque Repère, Bigard, Bio Village, Charal, F...",Viande bovine,Boucherie,<!DOCTYPE html> <!--[if lte IE 9]><html clas...,Ajouter Produit\n15.95 €\n21.27 € / kg\nViande...,https://fd4-courses.leclercdrive.fr/magasin-12...
1,"[Marque Repère, Férial, Tendriade]",Veau,Boucherie,<!DOCTYPE html> <!--[if lte IE 9]><html clas...,Ajouter Produit\n10.49 €\n13.45 € / kg\nVeau :...,https://fd4-courses.leclercdrive.fr/magasin-12...
2,[],Agneau,Boucherie,<!DOCTYPE html> <!--[if lte IE 9]><html clas...,Ajouter Produit\n6.34 €\n22.64 € / kg\nAgneau ...,https://fd4-courses.leclercdrive.fr/magasin-12...
3,"[Marque Repère, Férial]",Porc,Boucherie,<!DOCTYPE html> <!--[if lte IE 9]><html clas...,Ajouter Produit\n3.21 €\n18.88 € / kg\nCôte pr...,https://fd4-courses.leclercdrive.fr/magasin-12...
4,"[Marque Repère, Bio Village, Charal, Férial, S...",Viande hachée et Farce,Boucherie,<!DOCTYPE html> <!--[if lte IE 9]><html clas...,Ajouter Produit\n3.40 €\n17.00 € / kg\nSteak h...,https://fd4-courses.leclercdrive.fr/magasin-12...
5,"[Marque Repère, Bigard, Bio Village, Férial, S...",Saucisses et Grillades,Boucherie,<!DOCTYPE html> <!--[if lte IE 9]><html clas...,Ajouter Produit\n3.96 €\n14.40 € / kg\nSauciss...,https://fd4-courses.leclercdrive.fr/magasin-12...
6,"[Marque Repère, Charal, Férial]",Sauces,Boucherie,<!DOCTYPE html> <!--[if lte IE 9]><html clas...,Ajouter Produit\n1.69 €\n14.08 € / kg\nSauce a...,https://fd4-courses.leclercdrive.fr/magasin-12...
7,"[Marque Repère, Autour de Lise, Le Gaulois, Lo...",Poulets,Volailles et Gibiers,<!DOCTYPE html> <!--[if lte IE 9]><html clas...,Ajouter Produit\n6.69 €\n5.35 € / kg\nPoulet b...,https://fd4-courses.leclercdrive.fr/magasin-12...
8,"[Marque Repère, Le Gaulois, Maitre Coq, Ronsar...",Dindes,Volailles et Gibiers,<!DOCTYPE html> <!--[if lte IE 9]><html clas...,Ajouter Produit\n7.10 €\n11.83 € / kg\nRôti fi...,https://fd4-courses.leclercdrive.fr/magasin-12...
9,"[Marque Repère, Montfort, Nos régions ont du t...",Canards et Cailles,Volailles et Gibiers,<!DOCTYPE html> <!--[if lte IE 9]><html clas...,Ajouter Produit\n2.77 €\n7.91 € / kg\nCuisse d...,https://fd4-courses.leclercdrive.fr/magasin-12...


In [1496]:
import html
pages = []
for index, row in data.iterrows():
    soup = row.soup
    page = html.unescape(soup.text)
    page.find('objElement')
    page.find('lstAssocFiltreProduit')
    page = page[(page.find('objElement')-2):(page.find('lstAssocFiltreProduit')-2)]
    page = page.split("objElement")
    pages.append({"shelter_child": row.shelter_child,
                  "shelter_parent": row.shelter_parent,
                  "brands": row.brands,
                  "page": page[1:-1]
                })

In [1497]:
products = []
for page in pages:
    for content in page['page']:
        item = content.split(",")
        for line in item:
            if "sLibelleLigne1" in line:
                description1 = line[18:-1]
            if "sLibelleLigne2" in line:
                description2 = line[18:-1]
            if "sPrixUnitaire" in line:
                price = line[17:-1]
            if "sPrixParUniteDeMesure" in line:
                price_per_unit = line[25:-1]
            if "sPrixPromo" in line:
                promo_price = line[14:-1]
            if "sUrlVignetteProduit" in line:
                image_url = line[23:-1]
            if "sOrigineZoneDepeche" in line:
                origin = line[23:-1]
            if "sUrlPageProduit" in line:
                url = line[19:-1]
        description = description1 + " " + description2
        if len(page['brands']) > 0:
            brand = ''.join([brand for brand in page['brands'] if brand in description])
        else:
            brand = ""
                
        products.append({"shelter_parent": page['shelter_parent'],
                             "shelter_child": page['shelter_child'],  
                             "description":description, 
                             "brand": brand,
                             "price": price,
                             "price_per_unit":price_per_unit,
                             "promo_price": promo_price,
                             "origin":origin,
                             "image_url": image_url,                    
                             "url": url
                            })

products

[{'shelter_parent': 'Boucherie',
  'shelter_child': 'Viande bovine',
  'description': 'Viande bovine Rumsteak***  Bio  A rôtir - 750g',
  'brand': '',
  'price': '15.95 €',
  'price_per_unit': '21.27 € / kg',
  'promo_price': '15.95 €',
  'origin': 'France',
  'image_url': 'https://fd4-photos.leclercdrive.fr/image.ashx?id=727629&use=l&cat=p&typeid=i',
  'url': 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/fiche-produits-16506-Viande-bovine-Rumsteak-Bio-.aspx'},
 {'shelter_parent': 'Boucherie',
  'shelter_child': 'Viande bovine',
  'description': 'Viande bovine : Rôti** Charolaise - 750g',
  'brand': '',
  'price': '13.38 €',
  'price_per_unit': '17.84 € / kg',
  'promo_price': '13.38 €',
  'origin': 'France',
  'image_url': 'https://fd4-photos.leclercdrive.fr/image.ashx?id=744670&use=l&cat=p&typeid=i',
  'url': 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/fiche-produits-38900-Viande-bovine-Roti.aspx'},
 {'shelter_parent': 'Boucherie',
  'shelter_child

In [1498]:
product_data = pd.DataFrame(products)
product_data

export_csv = product_data.to_csv (r'/Users/vincentsalamand/code/jedha/products_leclerc.csv', index = None, header=True)


,brand,description,image_url,origin,price,price_per_unit,promo_price,shelter_child,shelter_parent,url
0,,Viande bovine Rumsteak*** Bio A rôtir - 750g,https://fd4-photos.leclercdrive.fr/image.ashx?...,France,15.95 €,21.27 € / kg,15.95 €,Viande bovine,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...
1,,Viande bovine : Rôti** Charolaise - 750g,https://fd4-photos.leclercdrive.fr/image.ashx?...,France,13.38 €,17.84 € / kg,13.38 €,Viande bovine,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...
2,,Viande bovine 1x rôti ** 600g,https://fd4-photos.leclercdrive.fr/image.ashx?...,France,10.11 €,16.85 € / kg,10.11 €,Viande bovine,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...
3,,Viande bovine 2x bavette Aloyau *** à griller ...,https://fd4-photos.leclercdrive.fr/image.ashx?...,France,6.64 €,23.71 € / kg,6.64 €,Viande bovine,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...
4,Charal,Bavette d'Aloyau *** Charal 2x130g,https://fd4-photos.leclercdrive.fr/image.ashx?...,,6.90 €,26.54 € / kg,6.90 €,Viande bovine,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...
5,Férial,Bavette Viande bovine Férial Aloyau ***à gril...,https://fd4-photos.leclercdrive.fr/image.ashx?...,,5.54 €,19.79 € / kg,5.54 €,Viande bovine,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...
6,,Viande bovine Bio Entrecôte *** x2 - 340g,https://fd4-photos.leclercdrive.fr/image.ashx?...,,10.56 €,31.06 € / kg,10.56 €,Viande bovine,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...
7,,Viande bovine 2x entrecôte *** à griller - 340g,https://fd4-photos.leclercdrive.fr/image.ashx?...,France,7.96 €,23.41 € / kg,7.96 €,Viande bovine,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...
8,,Viande bovine Charolaise 2x entrecôte *** à gr...,https://fd4-photos.leclercdrive.fr/image.ashx?...,France,9.80 €,27.22 € / kg,9.80 €,Viande bovine,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...
9,,Viande bovine 1x entrecôte*** Epaisse à grille...,https://fd4-photos.leclercdrive.fr/image.ashx?...,f'rance,6.99 €,23.30 € / kg,6.99 €,Viande bovine,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...


In [1506]:
dada = product_data[product_data.shelter_child == 'Légumes' ]
dada


,brand,description,image_url,origin,price,price_per_unit,promo_price,shelter_child,shelter_parent,url
423,,Salade : Laitue 1 p,https://fd4-photos.leclercdrive.fr/image.ashx?...,France,1.41 €,3.05 € / kg,1.41 €,Légumes,Légumes,https://fd4-courses.leclercdrive.fr/magasin-12...
424,,Laitue Iceberg 1p,https://fd4-photos.leclercdrive.fr/image.ashx?...,Espagne,1.69 €,3.05 € / kg,1.69 €,Légumes,Légumes,https://fd4-courses.leclercdrive.fr/magasin-12...
425,,Salade : Batavia 1p,https://fd4-photos.leclercdrive.fr/image.ashx?...,France,1.41 €,3.05 € / kg,1.41 €,Légumes,Légumes,https://fd4-courses.leclercdrive.fr/magasin-12...
426,,Salade : Feuille de chêne 1 p,https://fd4-photos.leclercdrive.fr/image.ashx?...,France,1.41 €,3.05 € / kg,1.41 €,Légumes,Légumes,https://fd4-courses.leclercdrive.fr/magasin-12...
427,,Salade : Feuille de chêne brune 1 p,https://fd4-photos.leclercdrive.fr/image.ashx?...,France,1.41 €,3.05 € / kg,1.41 €,Légumes,Légumes,https://fd4-courses.leclercdrive.fr/magasin-12...
428,,Salade : Coeur de romaine 2x1 p,https://fd4-photos.leclercdrive.fr/image.ashx?...,Espagne,2.50 €,1.25 € / p,2.50 €,Légumes,Légumes,https://fd4-courses.leclercdrive.fr/magasin-12...
429,,Salade : Sucrine 3 p,https://fd4-photos.leclercdrive.fr/image.ashx?...,Espagne,1.49 €,0.50 € / p,1.49 €,Légumes,Légumes,https://fd4-courses.leclercdrive.fr/magasin-12...
430,Bio Village,Courgette Bio Village 1kg,https://fd4-photos.leclercdrive.fr/image.ashx?...,' :espagne,3.49 €,3.49 € / kg,3.49 €,Légumes,Légumes,https://fd4-courses.leclercdrive.fr/magasin-12...
431,,Courgettes Bio 500g,https://fd4-photos.leclercdrive.fr/image.ashx?...,' italie,2.89 €,5.78 € / kg,2.89 €,Légumes,Légumes,https://fd4-courses.leclercdrive.fr/magasin-12...
432,,Courgette 1kg,https://fd4-photos.leclercdrive.fr/image.ashx?...,Espagne,1.49 €,1.49 € / kg,1.49 €,Légumes,Légumes,https://fd4-courses.leclercdrive.fr/magasin-12...
